# 1. 什么是MySQL多实例？

简单的说，就是**一台机器上开启多个不同的服务端口（如3306,3307），运行多个MySQL服务进程，这些进程通过不同的socket监听不同的服务端口来提供各自的服务**。

这些MySQL多实例共用一套MySQL安装程序，使用
- 不同（也可以相同）的my.cnf配置文件、启动程序文件，数据文件。

本次的案例采用共用一套安装程序，使用不同的配置文件、不同的启动程序、不同的数据文件的mysql多实例。

在提供服务的时候，多实例MySQL在逻辑上看来是各自独立的，多个实例的自身是根据配置文件对应的设置值，来取得服务器的相关硬件资源多少的。

做一个形象的比喻，MySQL多实例相当于房子的多个卧室一样，每个实例可以看作是一个卧室，整个服务器就是一套房子，服务器的硬件资源可以看作是房子的卫生间、厨房一样是房子的公共资源。就像是北漂住在一套房子里，肯定要共享资源。其实很多服务都可以有多实例如nginx,apache,redias等。

# 2. MySQL多实例的作用与存在的问题

1. 有效利用服务器资源

当单个服务器资源有剩余时，可以充分利用剩余的资源提供更多的服务。一般服务器资源占用率应该到60%-70%。

2. 节约服务器资源

当公司资金比较紧张，但是数据库又需要各自尽量独立提供服务，而且需要主从同步技术时多实例就再好不过了。

3. 资源互相抢占问题

当某个实例并发很高或者慢查询时，整个实例会消耗整个服务器更多的内存、cpu、磁盘IO资源，导致服务器上的其他实例提供服务的质量下降，这就相当于大家住在一个房子的不同卧室一样，早晨起来要上班，都要刷牙、洗脸等等。这样卫生间就会长期占用，其他人要等待一样的道理。

# 3. MySQL多实例生产应用场景
### 1. 资金紧张型公司的选择    
当公司业务访问量不太大，又不舍得花钱，但又希望不同业务的数据库服务各自尽量独立的听过服务互相不受影响，而且，需要**主从同步**进行等技术**提供备份或读写分离服务**，多实例就再好不过了。
如：   
可以通过3台服务器部署6-9个实例，交叉做主从同步备份及读写分离，实现6-9台服务器才有的效果。这里要强调的是，所谓的尽量独立是相对的。

### 2. 并发访问不是特别大的业务
当公司业务访问量不太大的时候，服务器的资源基本都是浪费的，这时就很适合多实例的应用，如果对SQL语句优化做的比较好，MySQL多实例一个很值得使用的技术，即使并发很大，合理分配好系统资源，也不会有太大问题。

### 3. 门户网应用MySQL多实例场景
 
百度搜索引擎的数据库就是多实例，一般是从库，48核，内存96G，跑3-4个实例，sina网也是用的多实例，内存48G左右。门户网站使用多实例的目的是配硬件好的服务器节省IDC机柜空间，同时，跑多实例让硬件资源不浪费。

# 4. mysql多实例常见配置方案

- 1. 通过多个配置文件及多个启动程序来实现多实例 
生产中比较推荐此方法，本随笔中也将采取此方法.

- 2. 单一配置文件方案即一个配置文件里写多个实例的内容
通过**mysqld_muti参数**实现，此方法耦合性太高，可参见 http://blog.51cto.com/suifu/1850560

## 1. 多配置文件部署方案

### 多实例MySQL详细步骤
#### 1.1 建立MySQL多实例的条件
- 安装好MySQL安装依赖的包

yum -y install ncurses-devel

yum -y install libaio-devel

- 选择二进制方式安装MySQL5.5,或者源码编译安装MySQL

#### 1.2 建立MySQL账号
首先以root身份登录到linux系统中，然后执行如下命令创建mysql组及用户

[root@mysql ~]# groupadd mysql

[root@mysql ~]# useradd -s /sbin/nologin -g mysql -M mysql

Useradd命令参数简要说明：

- -s /sbin/nologin 表示禁止该用户登录，加强安全。

- -g mysql 指定属于mysql组

- -M表示不创建用户家目录

#### 1.3 下载MySQL软件
通过rz或sftp等命令从本地上传mysql软件包到linux服务器

[root@mysql ~]# mkdir -p /application/tools/                     ß建立存放mysql软件包的目录

[root@mysql ~]# cd /application/tools/            ß进入软件包目录

[root@mysql tools]# ll mysql-5.5.32-linux2.6-x86_64.tar.gz   ß查看上传软件包

-rw-r--r-- 1 root root 186722932 Jun 20  2013 mysql-5.5.32-linux2.6-x86_64.tar.gz

#### 1.4 安装MYSQL软件
1. 解压软件包，移动到规范的软件安装目录下

2. 生成不带版本号的软链接/application/mysql，即mysql安装路径。

[root@mysql tools]# tar xf mysql-5.5.32-linux2.6-x86_64.tar.gz  ß解包

[root@mysql tools]# mv mysql-5.5.32-linux2.6-x86_64 /application/ß移动到规范的软件安装目录下

[root@mysql tools]#ln -s /application/mysql-5.5.32/ /application/mysql  ß生成不带版本链接/application/mysql，即mysql安装路径

#### 1.5 创建MySQL多实例的数据文件目录
 

下面是多实例的目录结构：

#### 1.6 创建MySQL多实例的配置文件my.cnf

创建MySQL多实例的配置文件，vim /data/3307/my.cnf，内容如下:

#### 1.7 创建多实例启动文件的脚本

多实例启动文件的启动mysql的服务实质：

多实例启动文件的停止mysql服务实质：

平滑停止数据库。

##### 创建MySQL多实例的启动文件，vim /data/3307/mysql ，内容如下：

#### 1.8 配置MySQL多实例的文件权限

#### 1.9 初始化MySQL多实例的数据库文件

查看3307目录下data目录，可以看到多出了不少的数据文件



2、为什么要初始化数据库？

初始化的主要目的就是创建基础的数据库文件，例如：生成mysql库表等。

初始化后/data/{3306,3307}/data目录下就会有mysql等数据文件。

#### 1.10 启动MySQL多实例数据库，检查看到3307的端口已经启动

----

### MySQL故障排错多实例数据库

如果mysql没有起来排查方法如下：

1. 如果发现没有显示MySQL的端口，请等待几秒mysql服务启动会有点慢。
2. 如果还是不行，请查看错误日志，错误日志路径在my.cnf配置文件路径下面。    
   grep log-error /data/3306/mysql_linzhongniaao3306.err|tail -1
3. 细看所有执行命令返回的输出，不要忽略关键的输出结果。
4. 查看系统的/var/log/messages。
5. 如果是关联服务，要同时查看相关服务的LOG。

### Mysql多实例登录

Mysql安装完成后，默认情况下管理员账号root是无密码的。登录不同的mysql数据库需要指定不同库的sock文件。如果是远程的话就指定IP地址。

-----

### MySQL安全配置

为root增加密码

设置完数据路密码登录数据库的方法

### 为mysql多实例启动脚本修改执行权限，只允许root用户有执行权限